In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
hear = pd.read_excel('hearing_impairment.xlsx')  #набор данных о анализах, имт и болезнях пациентов
hear

,Пол,Возраст,Профессия (должность),Классификация труда,Тяжесть,Нагрев,Шум,Стаж (количество лет),"Аудиометрия (справа) НЧ, дБ","Аудиометрия (справа) ВЧ, дБ","Аудиометрия (слева) НЧ, дБ","Аудиометрия (слева) ВЧ, дБ",Наруш слух,"Глюкоза, ммоль/л","Общий холестерин, ммоль/л","ЛПНП, ммоль/л","ЛПВП, ммоль/л",Индекс атерогенности,Жалобы на боль в анамнезе (ШОП)
0,0,30,токарь,3,1.0,0.0,1.0,15.0,10.0,10,10,10,0,5.19,3.23,1.55,1.06,2.05,0.0
1,1,47,шлифовщик,2,1.0,0.0,1.0,NaN,15.0,15,15,15,0,4.14,4.21,2.83,1.30,2.24,1.0
2,0,26,оператор,1,0.0,0.0,0.0,12.0,10.0,10,10,10,0,NaN,NaN,NaN,NaN,NaN,0.0
3,1,54,машинист,2,0.0,0.0,0.0,NaN,15.0,30,15,30,1,5.08,4.91,3.42,1.17,3.20,0.0
4,1,42,штамповщик,3,0.0,0.0,1.0,10.0,10.0,10,10,10,0,4.82,5.82,3.66,1.09,4.34,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,0,48,такелажник,4,1.0,0.0,1.0,NaN,10.0,20,10,30,1,NaN,NaN,NaN,NaN,NaN,0.0
478,0,52,слесарь,3,NaN,NaN,1.0,30.0,15.0,15,15,15,0,5.19,6.19,4.16,0.94,5.59,0.0
479,0,63,токарь,3,1.0,NaN,1.0,40.0,15.0,40,15,40,1,4.67,4.32,2.80,1.28,2.37,1.0
480,0,50,наладчик,2,1.0,NaN,1.0,NaN,15.0,15,15,15,0,4.85,7.33,3.07,2.04,2.59,0.0


In [ ]:
hear['Стаж (количество лет)'] = hear['Стаж (количество лет)'].fillna(int(hear['Стаж (количество лет)'].mean()))
hear['Стаж (количество лет)']

,Стаж (количество лет)
0,15.0
1,23.0
2,12.0
3,23.0
4,10.0
...,...
477,23.0
478,30.0
479,40.0
480,23.0


In [ ]:
hear['Тяжесть'] = hear['Тяжесть'].fillna(int(hear['Тяжесть'].mean()))
hear['Нагрев'] = hear['Нагрев'].fillna(int(hear['Нагрев'].mean()))
hear['Шум'] = hear['Шум'].fillna(int(hear['Шум'].mean()))
hear

,Пол,Возраст,Профессия (должность),Классификация труда,Тяжесть,Нагрев,Шум,Стаж (количество лет),"Аудиометрия (справа) НЧ, дБ","Аудиометрия (справа) ВЧ, дБ","Аудиометрия (слева) НЧ, дБ","Аудиометрия (слева) ВЧ, дБ",Наруш слух,"Глюкоза, ммоль/л","Общий холестерин, ммоль/л","ЛПНП, ммоль/л","ЛПВП, ммоль/л",Индекс атерогенности,Жалобы на боль в анамнезе (ШОП)
0,0,30,токарь,3,1.0,0.0,1.0,15.0,10.0,10,10,10,0,5.19,3.23,1.55,1.06,2.05,0.0
1,1,47,шлифовщик,2,1.0,0.0,1.0,23.0,15.0,15,15,15,0,4.14,4.21,2.83,1.30,2.24,1.0
2,0,26,оператор,1,0.0,0.0,0.0,12.0,10.0,10,10,10,0,NaN,NaN,NaN,NaN,NaN,0.0
3,1,54,машинист,2,0.0,0.0,0.0,23.0,15.0,30,15,30,1,5.08,4.91,3.42,1.17,3.20,0.0
4,1,42,штамповщик,3,0.0,0.0,1.0,10.0,10.0,10,10,10,0,4.82,5.82,3.66,1.09,4.34,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,0,48,такелажник,4,1.0,0.0,1.0,23.0,10.0,20,10,30,1,NaN,NaN,NaN,NaN,NaN,0.0
478,0,52,слесарь,3,0.0,0.0,1.0,30.0,15.0,15,15,15,0,5.19,6.19,4.16,0.94,5.59,0.0
479,0,63,токарь,3,1.0,0.0,1.0,40.0,15.0,40,15,40,1,4.67,4.32,2.80,1.28,2.37,1.0
480,0,50,наладчик,2,1.0,0.0,1.0,23.0,15.0,15,15,15,0,4.85,7.33,3.07,2.04,2.59,0.0


In [ ]:
hear['Глюкоза, ммоль/л'] = hear['Глюкоза, ммоль/л'].fillna(int(hear['Глюкоза, ммоль/л'].mean()))
hear['Общий холестерин, ммоль/л'] = hear['Общий холестерин, ммоль/л'].fillna(int(hear['Общий холестерин, ммоль/л'].mean()))
hear['ЛПНП, ммоль/л'] = hear['ЛПНП, ммоль/л'].fillna(int(hear['ЛПНП, ммоль/л'].mean()))
hear['ЛПВП, ммоль/л'] = hear['ЛПВП, ммоль/л'].fillna(int(hear['ЛПВП, ммоль/л'].mean()))
hear['Индекс атерогенности'] = hear['Индекс атерогенности'].fillna(int(hear['Индекс атерогенности'].mean()))
hear['Жалобы на боль в анамнезе (ШОП)'] = hear['Жалобы на боль в анамнезе (ШОП)'].fillna(int(hear['Жалобы на боль в анамнезе (ШОП)'].mean()))


In [ ]:
hear_nan = hear.isna().any()
print("Столбцы с NaN значениями:")
print(hear_nan[hear_nan == True].index.tolist())

Столбцы с NaN значениями:
[]


In [ ]:
hear = hear.drop('Профессия (должность)', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [ ]:
X = hear.drop('Наруш слух', axis=1)
y = hear['Наруш слух']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучение модели
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Точность
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность: {accuracy:.3f}")

# Отчет о классификации
report = classification_report(y_test, y_pred)
print("\nОтчет о классификации:\n")
print(report)

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
print("\nМатрица ошибок:\n")
print(cm)

# ROC-AUC (для многоклассовой задачи)
auc = roc_auc_score(y_test, y_pred)
print(f"\nROC-AUC: {auc:.3f}")

Точность: 1.000

Отчет о классификации:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        24

    accuracy                           1.00        97
   macro avg       1.00      1.00      1.00        97
weighted avg       1.00      1.00      1.00        97


Матрица ошибок:

[[73  0]
 [ 0 24]]

ROC-AUC: 1.000
